In [1]:
%matplotlib inline
import os
from IPython.display import Image
from IPython.display import display
import numpy as np
import pandas as pd
from scipy import interp
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import auc, confusion_matrix,  f1_score, precision_score, recall_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from cycler import cycler
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
plt = matplotlib.pyplot

### Create team

In [2]:
df_team = pd.read_csv("team\\team_esp.csv")


In [3]:
df_team

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Atlético Madrid,38,26,8,4,67,25,42,86,52.4,32.7,19.6,0.52,NaN,Luis Suárez - 21,Jan Oblak,→ UEFA Champions League via league finish
1,2,Real Madrid,38,25,9,4,67,28,39,84,61.6,36.5,25.2,0.66,NaN,Karim Benzema - 23,Thibaut Courtois,→ UEFA Champions League via league finish
2,3,Barcelona,38,24,7,7,85,38,47,79,78.9,39.6,39.3,1.03,NaN,Lionel Messi - 30,Marc-André ter Stegen,→ UEFA Champions League via league finish
3,4,Sevilla,38,24,5,9,53,33,20,77,50.9,35.0,15.9,0.42,NaN,Youssef En-Nesyri - 18,Yassine Bounou,→ UEFA Champions League via league finish
4,5,Real Sociedad,38,17,11,10,59,38,21,62,60.4,36.6,23.8,0.63,NaN,Alexander Isak - 17,Álex Remiro,→ UEFA Europa League via league finish
5,6,Betis,38,17,10,11,50,50,0,61,46.0,45.5,0.5,0.01,NaN,Borja Iglesias - 11,Claudio Bravo,→ UEFA Europa League via league finish 1
6,7,Villarreal,38,15,13,10,60,44,16,58,58.0,43.5,14.5,0.38,253.0,Gerard Moreno - 23,Sergio Asenjo,NaN
7,8,Celta Vigo,38,14,11,13,55,57,-2,53,47.7,49.5,-1.8,-0.05,91.0,Iago Aspas - 14,Iván Villar,NaN
8,9,Granada,38,13,7,18,47,65,-18,46,39.4,53.7,-14.3,-0.38,NaN,Roberto Soldado - 9,Rui Silva,NaN
9,10,Athletic Club,38,11,13,14,46,42,4,46,44.4,36.3,8.1,0.21,NaN,Álex Berenguer - 8,Unai Simón,NaN


In [4]:
df_team.values[:,1]

array(['Atlético Madrid', 'Real Madrid', 'Barcelona', 'Sevilla',
       'Real Sociedad', 'Betis', 'Villarreal', 'Celta Vigo', 'Granada',
       'Athletic Club', 'Osasuna', 'Cádiz', 'Valencia', 'Levante',
       'Getafe', 'Alavés', 'Elche', 'Huesca', 'Valladolid', 'Eibar'],
      dtype=object)

### Create player dataset

In [6]:
from glob import glob
files = glob("player_data_seriea/*.csv")

In [7]:
test=pd.read_csv(files[100])
pd.read_csv(files[100])


,Unnamed: 0,Age,Squad,Country,Comp,LgRank,90s,Cmp,Att,Cmp%,...,PrgDist.1,Prog.1,1/3.1,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.2
0,0,23,Inter,it ITA,1. Serie A,2nd,8.0,422,515,81.9,...,1089,60,15,5,12,14,514,449,87.4,71
1,1,24,Inter,it ITA,1. Serie A,1st,6.7,351,426,82.4,...,875,37,13,4,14,9,419,378,90.2,38


In [8]:
test.values[:,6]


array([8.0, 6.7], dtype=object)

In [9]:
player_n=len(files)
player_n

619

In [11]:
def one_player(df_one):
    df_num=df_one.shape[0]
    one_player=np.zeros((1, 71))
    for i in range(0,df_num):
    #print(i)
        if df_one.values[i,4]=='1. Serie A' :  # only collect the Premier Leage data
            dfxx= df_one.values[i,6:]           # collect the data after the column "LgRank"
            one_player=np.row_stack((one_player,dfxx))
            player_team_one= df_one.values[i,2]
        #if i==df_num-1:
         #   player_team_one= df_one.values[i,2]
    one_player=np.delete(one_player, 0, 0)     # delete temporay row
    #one_player=np.delete(one_player, 71-15, 1) # move the column have NAN
    one_player=np.mean(one_player,axis=0)      # average the data for different seasons
    return one_player,player_team_one

In [12]:
player= np.zeros((1, 71))  #70 becuase that i delelete one NAN column
for i in range(0,player_n):   # 英超,532 players
    df_one = pd.read_csv(files[i])
    player_one,_= one_player(df_one)
    player= np.row_stack((player,player_one))
player=np.delete(player, 0, 0)     # delete temporay row

In [13]:
player.shape

(619, 71)

In [14]:
player_team= []  #70 becuase that i delelete one NAN column
for i in range(0,player_n):   # 英超,532 players
    df_one = pd.read_csv(files[i])
    _,player_team_one= one_player(df_one)
    player_team.append(player_team_one)


In [15]:
len(player_team)

619

In [16]:
player_team

['Inter',
 'Inter',
 'Juventus',
 'Juventus',
 'Juventus',
 'Juventus',
 'Juventus',
 'Juventus',
 'Genoa',
 'Juventus',
 'Juventus',
 'Juventus',
 'Inter',
 'Juventus',
 'Juventus',
 'Juventus',
 'Juventus',
 'Juventus',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Inter',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Inter',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Napoli',
 'Udinese',
 'Fiorentina',
 'Napoli',
 'Inter',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Inter',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Inter',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Lazio',
 'Inter',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 'Inter',
 'Roma',
 'Roma',
 'Roma',
 'Roma',
 '

In [34]:
np.save("player_team_italy.npy",player_team)

In [18]:
player

array([[35.5, 952.0, 1102.0, ..., 627.0, 99.7, 0.0],
       [19.9, 598.0, 827.0, ..., 623.0, 82.4, 75.0],
       [12.9, 1013.0, 1073.0, ..., 948.0, 96.3, 19.0],
       ...,
       [18.9, 667.0, 784.0, ..., 707.0, 79.0, 135.0],
       [8.8, 550.5, 599.5, ..., 506.5, 98.15, 3.5],
       [18.25, 926.5, 1116.5, ..., 1110.0, 86.05, 178.0]], dtype=object)

In [19]:
X=player
X=X.astype(np.float32)

In [20]:
X.shape

(619, 71)

In [21]:
id=np.argwhere(np.isnan(X)) #尋找缺值的地方
id2=id[:,1] # 有那些feature還有missing value  # of missing:132 
id3=id[:,0] # 含missing value的player

In [22]:
X[id3].shape

(234, 71)

In [23]:
np.unique(id2)

array([ 3,  8, 11, 14, 56, 69], dtype=int64)

In [24]:
X_del_miss=np.delete(X, id3, 0)   # delete players who have missing values #備用資料集

In [25]:
X_del_miss.shape

(458, 71)

#### 補值處理  by KNN

In [26]:
from sklearn.impute import KNNImputer
#X = [[1, 2, np.nan], [3, 4, 3], [np.nan, 6, 5], [8, 8, 7]]
imputer = KNNImputer(n_neighbors=11)

In [27]:
X_add_miss= imputer.fit_transform(X)


In [28]:
X_add_miss.shape

(619, 71)

In [29]:
X_add_miss[0].shape

(71,)

In [30]:
id_test=np.argwhere(np.isnan(X_add_miss))
id_test

array([], shape=(0, 2), dtype=int64)

In [33]:
np.save("X_add_miss_italy.npy",X_add_miss)


### Create club dataset

In [35]:
def get_index(lst=None, item=''):
    return [index for (index,value) in enumerate(lst) if value == item]

In [36]:
id=get_index(player_team, 'Chelsea')

In [37]:
id

[]

In [38]:
len(player_team)

581

In [39]:
club_feature = np.zeros((1, 781))
for i in range(0,20):
    team_name=df_team.values[:,1][i]
    id_club = get_index(player_team, team_name) # 把屬於這個球隊的球員id找出來 # player_team是所有球員的對應球隊列表
    num = len(id_club)
    player_feature=np.zeros((1, 71))
    player_time_arr=np.zeros((num))
    for j in range(0,num):
        player_time_arr[j]=X_add_miss[id_club[j]][0]#
    idd =np.argsort([player_time_arr])#由小到大
    iddr = idd[::-1]
    ids = iddr[0][0:11] # select most important 11 players   #[0]
    nums = len(ids)
    for ii in range(0,nums):
        tmp=X_add_miss[ids[ii]]##<-------------------要改的是idclub
        tmp=tmp.reshape((1, 71))
        player_feature= np.column_stack((player_feature,tmp))
    #player_featurex= player_feature test
    tmpp=[i for i in range(71)]
    player_feature = np.delete(player_feature, tmpp)      # delete temporay row
    num_fe = len(player_feature)
    player_feature = player_feature.reshape((1, num_fe))
    if player_feature.shape[1]==781:
        #print("exe")
        club_feature= np.row_stack((club_feature,player_feature))
club_feature=np.delete(club_feature, 0, 0)     # delete temporay row

    

d = np.array([4,3,2,1])
d = np.pad(d,(0,5),'constant',constant_values=(0,0))
print(d)

In [40]:
club_feature.shape

(20, 781)

In [41]:
np.save("regression_X_esp.npy",club_feature)


In [42]:
df_team.values[:,1]

array(['Atlético Madrid', 'Real Madrid', 'Barcelona', 'Sevilla',
       'Real Sociedad', 'Betis', 'Villarreal', 'Celta Vigo', 'Granada',
       'Athletic Club', 'Osasuna', 'Cádiz', 'Valencia', 'Levante',
       'Getafe', 'Alavés', 'Elche', 'Huesca', 'Valladolid', 'Eibar'],
      dtype=object)

### 跳掉leeds united ,手動輸入控球率

In [43]:
Y=np.array([52.4,59.7,65.8,61.2,55.1,53.9,55.6,53.7,41.0,49.8,43.1,34.3,46.9,52.4,42.8,42.7,47.6,48.3,45.1,49.6])

In [44]:
Y.shape

(20,)

In [45]:
Y=Y.reshape((20, 1))


In [46]:
np.save("regression_Y_esp.npy",Y)
